# Advent of Code 2020, Python edition

Annotated solutions in Python.

Note that part of the charm of AoC is that every user (or at least groups of users) gets their own unique data set. Some of the solutions below exploit quirks in my particular data set, and so may conceivably not work for the general case.

In [52]:
from collections import Counter, defaultdict, deque
from copy import deepcopy
from dataclasses import dataclass
from functools import lru_cache, reduce
from itertools import accumulate, count, chain, combinations, count, product, takewhile
from math import ceil, prod
from operator import mul
import re
import sys

### Day 1: Report Repair
https://adventofcode.com/2020/day/1

In [20]:
def day01(d, size):
    def finder(x): return sum(x) == 2020
    return prod(next(filter(finder, combinations(d, size))))

with open('data/2020/day01.txt') as f:
    data = [int(l) for l in f]

p1 = day01(data, 2)
p2 = day01(data, 3)

assert p1 == 73371
assert p2 == 127642310
print(p1, p2)

73371 127642310


### Day 2: Password Philosophy
https://adventofcode.com/2020/day/2

In [21]:
def part1(d):
    count = sum(map(lambda c: c == d[2], d[-1]))
    return int(d[0]) <= count <= int(d[1])

def part2(d):
    return (d[2] == d[-1][int(d[0])-1]) != (d[2] == d[-1][int(d[1])-1])

with open('data/2020/day02.txt') as f:
    data = [re.split(r'[ :-]', l) for l in f]

p1 = sum(map(part1, data))
p2 = sum(map(part2, data))

assert p1 == 528
assert p2 == 497
print(p1, p2)

528 497


### Day 3: Toboggan Trajectory
https://adventofcode.com/2020/day/3

In [22]:
def day03(d, dy, dx):
    return sum('#'==d[dy*i][(dx*i)%len(d[0])] for i in range(ceil(len(d)/dy)))

with open('data/2020/day03.txt') as f:
    data = f.read().splitlines()

result = [day03(data, *v) for v in [[1, 1], [1, 3], [1, 5], [1, 7], [2, 1]]]

p1 = result[1]
p2 = prod(result)

assert p1 == 203
assert p2 == 3316272960
print(p1, p2)

203 3316272960


### Day 4: Passport Processing
https://adventofcode.com/2020/day/4

In [32]:
SYMTAB = {
    'eyr': lambda a: 2020 <= int(a) <= 2030,
    'iyr': lambda a: 2010 <= int(a) <= 2020,
    'byr': lambda a: 1920 <= int(a) <= 2002,
    'ecl': lambda a: a in {'amb', 'blu', 'brn', 'gry', 'grn', 'hzl', 'oth'},
    'cid': lambda a: True,
    'pid': lambda a: re.match(r'^\d{9}$', a),
    'hcl': lambda a: re.match(r'^#[a-f0-9]{6}$', a),
    'hgt': lambda a: re.match(r'^(((59|6[0-9]|7[0-6])in)|((1[5-8][0-9]|19[0-3])cm))$',a)
}

def part1(pp):
    return {'byr', 'iyr', 'eyr', 'hgt', 'hcl', 'ecl', 'pid'} <= set(pp.keys())

def part2(passports):
    count = 0
    for pp in passports:
        try:
            for check, arg in pp.items():
                if not (part1(pp) and SYMTAB[check](arg)):
                    raise Exception()
            count += 1
        except:        # Sorry. Convenience wins over pythonic
            pass

    return count

def parse_data(data):
    keyvals = {}
    for d in (re.findall(r'([^:\s]+):([^\s]+)', line) for line in data):
        if not d:
            yield keyvals
            keyvals = {}
            continue
        keyvals |= dict(d) # Recent pythons only
    yield keyvals

with open('data/2020/day04.txt') as f:
    passports = list(parse_data(f))

p1 = sum(part1(p) for p in passports)
p2 = part2(passports)

assert p1 == 256
assert p2 == 198
print(p1, p2)

256 198


### Day 5: Binary Boarding
https://adventofcode.com/2020/day/5

In [24]:
def SeatID(spec):
    return int(''.join({'F':'0','L':'0'}.get(l, '1') for l in spec), 2)

with open('data/2020/day05.txt') as f:
    data = [SeatID(l) for l in f.read().splitlines()]

p1 = max(data)
p2 = list(set(range(min(data), p1)) - set(data))[0]

assert p1 == 888
assert p2 == 522
print(p1, p2)

888 522


### Day 6: Custom Customs
https://adventofcode.com/2020/day/6

In [25]:
def gr(data):
    g = []
    for l in data:
        if l != '':
            g.append(l)
        else:
            yield g
            g = []
    yield g
    
with open('data/2020/day06.txt') as f:
    data = list(gr(f.read().splitlines()))

p1 = sum((len(set(''.join(l))) for l in data))
p2 = sum(len(set.intersection(*[set(s) for s in g])) for g in data)

assert p1 == 6416
assert p2 == 3050
print(p1, p2)

6416 3050


### Day 7: Handy Haversacks
https://adventofcode.com/2020/day/7

In [26]:
def parse(data):
    head, tail = data.split(' bags contain ')
    tail = tail.replace('no other bags.', '0 other bags.')
    contents = [
        re.findall(r'^(\d+)\s+(\w+\s\w+)', ss) for ss in tail.split(', ')
    ]
    return (head, [(int(w[0][0]), w[0][1]) for w in contents])

def invert(data):
    contained_in = defaultdict(set)
    for spec in data:
        for (_, name) in spec[1]:
            contained_in[name].add(spec[0])
    return contained_in

def vert(data):
    contains = defaultdict(dict)
    for spec in data:
        for (mag, name) in spec[1]:
            contains[spec[0]][name] = mag
    return contains

def part1(inverted):
    found = set()
    queue = list(inverted['shiny gold'])
    for item in queue:
        found.add(item)
        if item in inverted:
            queue.extend(list(inverted[item]))
    return len(found)

def part2(verted, key='shiny gold'):
    if key == 'other bags': return 0
    return sum(vv * (1 + part2(verted, kk)) for kk, vv in verted[key].items())

with open('data/2020/day07.txt') as f:
    data = [parse(l) for l in f]

p1 = part1(invert(data))
p2 = part2(vert(data))

assert p1 == 185
assert p2 == 89084
print(p1, p2)

185 89084


### Day 8: Handheld Halting
https://adventofcode.com/2020/day/8

In [27]:
@dataclass
class State:
    ip: int
    code: list
    args: list
    acc: int
    lines: list

    def halted(self):
        return self.ip >= len(self.code)
    
    def infloop(self):
        return 2 in self.lines
    
    def step(self):
        instr = self.code[self.ip]
        arg = self.args[self.ip]
        self.lines[self.ip] += 1
        if self.infloop():
            return 
        if instr == 'nop':
            self.ip += 1
        elif instr == 'acc':
            self.acc += arg
            self.ip += 1
        else:
            self.ip += arg
            
    def run(self):
        while not self.halted() and not self.infloop():
            self.step()
        return self.acc

def tweak(code):
    def repl(data, idx, to):
        data[idx] = to
        return data

    for idx, instr in enumerate(code):
        if instr == 'nop':
            yield repl(list(code), idx, 'jmp')
        elif instr == 'jmp':
            yield repl(list(code), idx, 'nop')

def part2(data):
    for code in tweak(data[0]):
        state = State(0, code, [int(a) for a in data[1]], 0, [0]*len(code))
        result = state.run()
        if state.halted():
            return result

with open('data/2020/day08.txt') as f:
    data = list(zip(*[l.split() for l in f]))

part1 = State(0, data[0], [int(a) for a in data[1]], 0, [0]*len(data[0]))

p1 = part1.run()
p2 = part2(data)

assert p1 == 1134
assert p2 == 1205
print(p1, p2)

1134 1205


### Day 9: Encoding Error
https://adventofcode.com/2020/day/9

For part 2, the sum of a stretch is the sum to the end, minus sum to the start. Outer product gives us all stretches, but we need to convert from single index back to start and end of the stretch. In APL, the outer product would have given us a 2D matrix of the correct dimensions, rather than a vector.

In [45]:
def find(data, idx, size):
    return data[idx] not in map(sum, combinations(data[idx-size:idx], 2))
    
def part1(data, win):
    for idx in range(win, len(data)):
        if find(data, idx, win): return data[idx]
    return -1

def part2(data, item):
    sums = list(accumulate(data))
    idx = [a[0]-a[1] for a in product(sums, sums)].index(item)
    values = data[1+idx%len(sums):idx//len(sums)+2]
    return min(values) + max(values)
        
with open('data/2020/day09.txt') as f:
    data = list(map(int, f))

p1 = part1(data, 25)
p2 = part2(data, p1)

assert p1 == 1639024365
assert p2 == 219202240
print(p1, p2)

1639024365 219202240


### Day 10: Adapter Array
https://adventofcode.com/2020/day/10

Part 2 -- a [Dynamic Programming](https://en.wikipedia.org/wiki/Dynamic_programming) solution. The cache only needs two slots.

In [113]:
def part1(data):
    diff = [data[i]-data[i-1] for i in range(1, len(data))] # Windowed reduction
    return prod(sum(d==c for d in diff) for c in [1, 3])

@lru_cache(maxsize=2)
def part2(v, data):
    if not data: return 1
    count = 0       # Valid solutions from this point
    for idx, item in enumerate(data):
        if item - v <= 3:
            count += part2(item, tuple(data[1+idx:]))
    return count

with open('data/2020/day10.txt') as f:
    data = sorted(list(map(int, f)))
data = [0, *data, 3 + data[-1]]

p1 = part1(data)
p2 = part2(0, tuple(data[1:]))

assert p1 == 2059
assert p2 == 86812553324672
print(p1, p2)

2059 86812553324672


Here's part 2 as a single reduction. Python folds left-right, unlike APL.

In [114]:
with open('data/2020/day10.txt') as f:
    data = sorted(list(map(int, f)))                # (⍋⌷¨⊂)⍎⍕⊃⎕NGET'data/2020/day10.txt'1

p2 = reduce(
    lambda acc, w:[*acc[1:], w*sum(acc)],           # {1↓⍵,⍺×+/⍵}
    [int(i in data) for i in range(1, data[-1]+1)], # ⌽data∊⍨1+⍳⊃⌽data
    [0, 0, 1]                                       # ⊂0 0 1
)[2]

assert p2 == 86812553324672
print(p2)

86812553324672


### Day 11: Seating System
https://adventofcode.com/2020/day/11

In [36]:
OFFSET = ((-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1))

def N8(data, pos):
    coords = ((pos[0] + t[0], pos[1] + t[1]) for t in OFFSET)
    return [
        data[y][x] for y, x in coords
        if 0 <= y < len(data) and 0 <= x < len(data[0])
    ]

def Transition(data, nf, pos, mag):
    elem = data[pos[0]][pos[1]]
    if elem == '.': return '.'
    c = Counter(nf(data, pos))
    if elem == 'L' and '#' not in c: return '#'
    if elem == '#' and c['#'] >= mag: return 'L'
    return elem

def day11(data, nf):
    return tuple(
        tuple(Transition(data, nf, (y, x), 4) for x in range(len(data[0])))
        for y in range(len(data)))

def C(data): return sum(e == '#' for e in chain(*data))

def N8_line_of_sight(data, pos):
    elems = []
    for delta in OFFSET:
        y, x = pos
        while True:
            y += delta[0]
            x += delta[1]
            try:
                elem = data[y][x]
            except IndexError:
                break
            if elem != '.':
                elems.append(elem)
                break
    return elems

with open('/Users/stefan/work/dyalog/AoCDyalog/data/2020/day11.txt') as f:
    data = tuple(tuple(l) for l in f.read().splitlines())

while (new := day11(data, N8)) != data:
    data = new

p1 = C(data)

assert p1 == 2344
print(p1)

# TODO: Part 2
# with open('/Users/stefan/work/dyalog/AoCDyalog/data/2020/day11.txt') as f:
#     data = tuple(tuple(l) for l in f.read().splitlines())
    
# # while (new := day11(data, N8_line_of_sight)) != data:
# #     data = new

# # p2 = C(data)
# # print(p2)

2344


### Day 12: Rain Risk
https://adventofcode.com/2020/day/12

In [135]:
# ---- PART 1 --------
def N(acc, mag): return [acc[0] - mag, acc[1], acc[2]]
def S(acc, mag): return N(acc, -mag)
def E(acc, mag): return [acc[0], acc[1] + mag, acc[2]]
def W(acc, mag): return E(acc, -mag)

def R(acc, mag):
    acc[2].rotate(-mag // 90)
    return [*acc[:2], acc[2]]

def L(acc, mag): return R(acc, -mag)

def F(acc, mag):
    dv = ((0, mag), (mag, 0), (0, -mag), (-mag, 0))[acc[2][0]]
    return [dv[0] + acc[0], dv[1] + acc[1], acc[2]]

def run(acc, item):
    return {'N': N, 'W': W, 'E': E, 'S': S, 'L': L, 'R': R, 'F': F,}[item[0]](acc, item[1])

# ---- PART 2 --------
def N2(acc, mag): return [acc[0]-mag, acc[1], acc[2], acc[3]]
def S2(acc, mag): return N2(acc, -mag)
def E2(acc, mag): return [acc[0], acc[1]+mag, acc[2], acc[3]]
def W2(acc, mag): return E2(acc, -mag)
def F2(acc, mag):
    d = [acc[0]*mag, acc[1]*mag]
    return [acc[0], acc[1], acc[2]+d[0], acc[3]+d[1]]
                          
def R2(acc, mag):
    if mag == 90:
        return [acc[1], -acc[0], acc[2], acc[3]]
    if mag == 180:
        return [-acc[0], -acc[1], acc[2], acc[3]]
    return [-acc[1], acc[0], acc[2], acc[3]]

def L2(acc, mag):
    return R2(acc, 360-mag)

def run2(acc, item):
    return {'N': N2, 'W': W2, 'E': E2, 'S': S2, 'L': L2, 'R': R2, 'F': F2,}[item[0]](acc, item[1])

# -------------------
with open('data/2020/day12.txt') as f:
    (cmd, arg) = list(zip(*[m[0] for m in [re.findall(r'^([FRLNWES])(\d+)$', l) for l in f]]))

data = tuple(zip(cmd, tuple(map(int, arg))))

result = list(reduce(run, data, [0, 0, deque([0, 1, 2, 3])]))[:2]
p1 = abs(result[0]) + abs(result[1])

result = list(reduce(run2, data, [-1, 10, 0, 0])) 
                                 
p2 = abs(result[2]) + abs(result[3])

assert p1 == 2879
assert p2 == 178986

print(p1, p2)

2879 178986


### Day 13: Shuttle Search
https://adventofcode.com/2020/day/13

Python implementation of the [Chinese Remainter Theorem](https://en.wikipedia.org/wiki/Chinese_remainder_theorem) taken verbatim from

https://rosettacode.org/wiki/Chinese_remainder_theorem#Python

In [139]:
def chinese_remainder(n, a):
    sum = 0
    prod = reduce(lambda a, b: a * b, n)
    for n_i, a_i in zip(n, a):
        p = prod // n_i
        sum += a_i * mul_inv(p, n_i) * p
    return sum % prod

def mul_inv(a, b):
    b0 = b
    x0, x1 = 0, 1
    if b == 1: return 1
    while a > 1:
        q = a // b
        a, b = b, a % b
        x0, x1 = x1 - q * x0, x0
    if x1 < 0: x1 += b0
    return x1

with open('data/2020/day13.txt') as f:
    data = f.read().splitlines()

timestamp = int(data[0])
buses = eval(f"[{data[1].replace('x', '0')}]")
n = [i for i in buses if i != 0]

p1 = next(delta * bus for delta in count() for bus in n
          if (timestamp + delta) % bus == 0)

idx = [buses.index(i) for i in n]
a = [n[j] - idx[j] + 1 for j in range(len(n))]
p2 = chinese_remainder(n, a) - 1

assert p1 == 410
assert p2 == 600691418730595
print(p1, p2)

410 600691418730595


### Day 14: Docking Data
https://adventofcode.com/2020/day/14

In [250]:
def gr(data):
    g = [data[0]]
    for line in data[1:]:
        if 'mask' in line:
            yield g
            g = [line]
        else:
            g.append(line)
    yield g

def numbers(string):
    return [int(n) for n in re.findall(r'(\d+)', string)]

def parse(group):
    ones = int(group[0][7:].replace('X', '0'), 2)
    zeros = int(
        group[0][7:].replace('0', 'Y').replace('X', '0').replace('1', '0').replace('Y', '1'), 2)
    exes = int(group[0][7:].replace('1', '0').replace('X', '1'))
    return ([ones, zeros, exes], [numbers(l) for l in group[1:]])

def part1(mem, group):
    for (addr, val) in group[1]:
        mem[addr] = (val | group[0][0]) & (group[0][1] ^ 0xfffffffff)

with open('data/2020/day14.txt') as f:
    data = f.read().splitlines()

mem = {}
for i in (parse(g) for g in gr(data)):
    part1(mem, i)

p1 = sum(mem.values())
print(p1)
assert p1 == 13496669152158

### TODO: PART2

13496669152158


### Day 15: Rambunctious Recitation
https://adventofcode.com/2020/day/15

This is the [Van Eck](https://www.youtube.com/watch?v=etMJxB-igrc) sequence.

In [248]:
def day15(numbers, target):
    spoken = [False] * target
    for i, v in enumerate(numbers):
        spoken[v] = i + 1
    last = 0
    for i in range(len(numbers) + 1, target):
        spoken[last], last = (i, 0) if not spoken[last] else (i, i - spoken[last])
    return last

In [249]:
data = [18, 8, 0, 5, 4, 1, 20]

p1 = day15(data, 2020)
print(f"{p1} ", end='')
assert p1 == 253

p2 = day15(data, 30000000)
assert p2 == 13710
print(p2)

253 13710


### Day 16: Ticket Translation
https://adventofcode.com/2020/day/16

In [35]:
def grouper(data):
    iterator = iter(data)
    while group := list(takewhile(lambda x: x, iterator)):
        yield group

def parse(data):
    rules = []
    for r in (re.findall(r'\d+', line) for line in data[0]):
        left = range(int(r[0]), int(r[1]) + 1)
        right = range(int(r[2]), int(r[3]) + 1)
        rules.append(list(chain(left, right)))

    ticket = eval(f"[{data[1][1]}]")
    near = [eval(f"[{line}]") for line in data[2][1:]]
    return rules, ticket, near

def part1(tickets, rules):
    return sum(n for n in chain(*tickets) if n not in set(chain(*rules)))

def aggregate_ticket_matches(all):
    table = [[0 for x in range(len(all[0][0]))] for y in range(len(all[0]))]
    for y in range(len(table)):
        for x in range(len(all[0][0])):
            table[y][x] = int(0 not in [all[z][y][x] for z in range(len(all))])
    return table

def match_rules(tickets, rules):
    all = []
    for ticket in tickets:
        matches = [[0 for x in range(len(tickets[0]))]
                   for y in range(len(rules))]
        for rid, rule in enumerate(rules):
            for fid, field in enumerate(ticket):
                if field in rule:
                    matches[fid][rid] = 1
        all.append(matches)
    return all

def resolve_fields(matching, rules):
    order = [-1] * len(rules)
    seen = set()
    while len(seen) != len(rules):
        for y, row in enumerate(matching):
            if sum(row) == 1:
                field = row.index(1)
                if field in seen:
                    continue
                seen.add(field)
                order[y] = field
                for yy in range(len(matching)):
                    if yy != y:
                        matching[yy][field] = 0
                break
    return order

def is_valid(t, valid):
    for n in t:
        if n not in valid:
            return False
    return True

def field_names(data):
    return [line.split(':')[0] for line in data[0]]

def part2(tickets, rules):
    valid = [
        ticket 
        for ticket in tickets 
        if is_valid(ticket, set(chain(*rules)))
    ]
    matches = match_rules(valid, rules)
    table = aggregate_ticket_matches(matches)
    return resolve_fields(table, rules)

with open('data/2020/day16.txt') as f:
    data = list(grouper(f.read().splitlines()))

rules, ticket, near = parse(data)
p1 = part1(near, rules)

order = part2(near, rules)
fields = field_names(data)
ordered_fields = [fields[x] for x in order]

p2 = prod(
    ticket[idx] 
    for idx, field in enumerate(ordered_fields) 
    if field.startswith('departure')
)

assert p1 == 29019
assert p2 == 517827547723
print(p1, p2)

29019 517827547723


### Day 17: Conway Cubes
https://adventofcode.com/2020/day/17

### Day 18: Operation Order
https://adventofcode.com/2020/day/18

This is a filthy and unhealthy hack.

Noting that we can evaluate strictly L-R by having multiplication and addition have the same precedence, we ceate a custom integer class that redefines - as ×, thus giving + and "×" the same precedence. We then tweak the data swapping out multiplication signs for subtraction and evaluate. 

Part 2 is more of the same.

In [16]:
class P1I():
    """
    Special kind of integer that thinks - is *
    """
    def __init__(self, val):
        self.val = val

    def __add__(self, other):  # + is still +
        return P1I(self.val + other.val)

    def __sub__(self, other):  # Make - be *
        return P1I(self.val * other.val)

    def __radd__(self, other):  # + is still +
        return self.__add__(other)

    def __rsub__(self, other):
        return self.__mul__(other)


class P2I():
    """
    Special kind of integer that thinks * is + and - is *
    """
    def __init__(self, val):
        self.val = val

    def __mul__(self, other):  # Make * be +
        return P2I(self.val + other.val)

    def __sub__(self, other):  # Make - be *
        return P2I(self.val * other.val)

    def __rmul__(self, other):
        return self.__add__(other)

    def __rsub__(self, other):
        return self.__mul__(other)


def part1(string):
    string = re.sub(r'(\d)', r'P1I(\1)', re.sub(r'\*', '-', string))
    return eval(string)


def part2(string):
    string = re.sub(r'\*', '-', string)
    string = re.sub(r'\+', '*', string)
    string = re.sub(r'(\d)', r'P2I(\1)', string)
    return eval(string)

with open('data/2020/day18.txt') as f:
    data = f.read().splitlines()

p1 = sum(part1(expr).val for expr in data)
p2 = sum(part2(expr).val for expr in data)

assert p1 == 7293529867931
assert p2 == 60807587180737
print(p1, p2)

7293529867931 60807587180737


For part 2, a different way is to bracket the expression fully according to the new rules, and then have Python evaluate it for us. For example, given

    5+(8×3+9+3×4×3)

we need to transform that to:

    5+(8×(3+9+3)×4×3)

We can solve this using some equally filthy regexing:

    Repeat each existing paranthesis:

    5+((8×((3+9+3))×4×3))

    Surround each digit with a pair of parentheses.

    (5)+(((8)×(((3)+(9)+(3)))×(4)×(3)))

    Remove the internal brackets in plus-groups

    (5+((8)×(((3+9+3)))×(4)×(3)))

In [17]:
def part2_regex(string):
    string = re.sub(r'\s+', '', string)
    string = re.sub(r'([)(])', r'\1\1', string)
    string = re.sub(r'(\d)', r'(\1)', string)
    string = re.sub(r'\)\+\(', '+', string)
    
    return eval(string)

In [18]:
p2r = part2_regex('5 + (8 * 3 + 9 + 3 * 4 * 3)')
assert p2r == 1445
print(p2r)

1445


### Day 20: Jurassic Jigsaw
https://adventofcode.com/2020/day/20

In [54]:
SIZE = 12
TILES = []

def flatten(lol):
    return [i for sublist in lol for i in sublist]

def grouper(data):
    iterator = iter(data)
    while group := list(takewhile(lambda x: x, iterator)):
        yield group
        
def transpose(data):
    return list(map(list, zip(*data)))

def rot(data):
    return [list(reversed(row)) for row in transpose(data)]

def flipX(data):
    return list(reversed(list(data)))

def trim(data):
    return [
        [data[y][x] for x in range(1, len(list(data[0])) - 1)]
        for y in range(1, len(data) - 1)
    ]

SYMM = [
    lambda d: d,
    lambda d: rot(d),
    lambda d: rot(rot(d)),
    lambda d: rot(rot(rot(d))),
    lambda d: flipX(d),
    lambda d: rot(flipX(d)),
    lambda d: rot(rot(flipX(d))),
    lambda d: rot(rot(rot(flipX(d))))
]

@dataclass
class Tile:
    id: int
    symmetry: int
    fit: list[int]  # N E S W
    data: list

    @classmethod
    def from_data(cls, data):
        d = [[i.replace('.', '0').replace('#', '1') for i in item]
             for item in data[1:]]
        obj = int(data[0][4:9])
        result = []
        for idx, sym in enumerate(SYMM):
            new = sym(d)
            result.append(cls(obj, idx, cls._edge_ids(new), new))
        return result

    @classmethod
    def _edge_ids(cls, data):
        trn = list(zip(*data))
        return [
            int(''.join(str(i) for i in data[0]), 2),   # N
            int(''.join(str(i) for i in trn[-1]), 2),   # E
            int(''.join(str(i) for i in data[-1]), 2),  # S
            int(''.join(str(i) for i in trn[0]), 2)     # W
        ]

class State:
    def __init__(self):
        self.next = (0, 0)  # y x  next position in tiles to be filled
        self.used = set()  # tiles already used
        # Note: indices into TILES array, not tile ids
        self.tiles = [[-1 for x in range(SIZE)] for y in range(SIZE)]
        self.found = False

    def value(self):
        nw = self.tiles[0][0]
        ne = self.tiles[0][-1]
        se = self.tiles[-1][-1]
        sw = self.tiles[-1][0]
        return prod([TILES[nw].id, TILES[ne].id, TILES[se].id, TILES[sw].id])

    def _find_n_w(self):
        """Find North and West neighbours"""
        N, W = None, None
        y = self.next[0] - 1
        if y >= 0 and self.next[1] >= 0:
            N = TILES[self.tiles[y][self.next[1]]]
        x = self.next[1] - 1
        if self.next[0] >= 0 and x >= 0:
            W = TILES[self.tiles[self.next[0]][x]]
        return N, W

    def _find_options(self, s, e):
        """
        Find all tiles - not already seen - that matches a northern
        tile with a southern edge s, and a western tile with an
        eastern edge e
        """
        options = []
        for idx, tile in enumerate(TILES):
            if tile.id in self.used:
                continue
            # N E S W
            south_fits = s == -1 or s == tile.fit[0]
            east_fits = e == -1 or e == tile.fit[3]
            if not self.used or (south_fits and east_fits):
                options.append(idx)  # Note: TILE index, not tile.id

        return options

    def _make_new_states(self, options):
        states = []
        for idx in options:
            state = deepcopy(self)

            # Mark the tile id as used. Only one of each id, regardless
            # of orientation
            state.used.add(TILES[idx].id)
            state.tiles[state.next[0]][state.next[1]] = idx

            y = state.next[0]
            x = state.next[1] + 1

            if x == SIZE:
                y += 1
                x = 0

            if y == SIZE:
                state.found = True
                return [state]

            state.next = (y, x)
            states.append(state)
            
        return states

    def neighbours(self):
        """Find valid tiles for position `next`."""
        N, W = self._find_n_w()

        SEdge, EEdge = -1, -1
        if N:
            SEdge = N.fit[2]
        if W:
            EEdge = W.fit[1]

        options = self._find_options(SEdge, EEdge)

        return self._make_new_states(options)

    def reconstruct(self):
        data = [TILES[x] for x in flatten(self.tiles)]
        return [data[i:i + SIZE] for i in range(0, len(data), SIZE)]

def bfs():
    queue = [State()]
    for state in queue:
        if state.found:
            return state
        queue.extend(state.neighbours())
        
    return False


def reassemble(best):
    mytiles = best.reconstruct()

    tilesize = len(mytiles[0][0].data[0]) - 2
    tilecount = len(mytiles)

    result = [['' for i in range(tilesize * tilecount)]
              for j in range(tilesize * tilecount)]

    for ty in range(tilecount):
        for tx in range(tilecount):
            data = trim(mytiles[ty][tx].data)
            for y, row in enumerate(data):
                for x, val in enumerate(row):
                    result[ty * tilesize + y][tx * tilesize + x] = val

    return result


def find_monster(img, monster):
    """Template matching. So many for-loops.."""
    monster_parts = 0
    hash_count = sum([i == '1' for i in chain(*img)])
    monster_count = sum([i == '#' for i in chain(*monster)])
    for y in range(len(img) - len(monster)):
        for x in range(len(img[0]) - len(monster[0])):
            found = 0
            for my in range(len(monster)):
                ypos = y + my
                for mx in range(len(monster[0])):
                    if monster[my][mx] == '#' and img[ypos][x + mx] == '1':
                        found += 1
            if found == monster_count:
                monster_parts += monster_count
                
    if monster_parts == 0:
        return 0

    return hash_count-monster_parts


with open('/Users/stefan/work/dyalog/AoCDyalog/data/2020/day20.txt') as f:
    data = list(grouper(f.read().splitlines()))

TILES = flatten([Tile.from_data(d) for d in data])

best = bfs()
p1 = best.value()
print(f"{p1} ", end='')

r = reassemble(best)

monster = ['..................#.', '#....##....##....###', '.#..#..#..#..#..#...']

p2 = None
for s in SYMM:
    p2 = find_monster(s(r), monster)
    if p2 != 0:
        break

print(p2)
assert p1 == 59187348943703
assert p2 == 1565

59187348943703 1565


### Day 21: Allergen Assessment
https://adventofcode.com/2020/day/21

In [50]:
def flatten(lol):
    return [i for sublist in lol for i in sublist]

def parse(data):
    a2i = {}
    foods = []
    allergens = set()

    for line in data:
        left, right = line.split(' (')
        ingr = left.split()
        all = right.replace(',', '').replace(')', '').split()[1:]
        foods.append(ingr)
        for a in all:
            allergens.add(a)
            if a not in a2i:
                a2i[a] = set(ingr)
            else:
                a2i[a] &= set(ingr)

    return a2i, foods, allergens

def resolve(allg):
    # Each allergen is present in a single ingredient, and
    # is guaranteed present somewhere.
    resolved = {}
    remove = set()
    while len(resolved) != len(allg):
        for a, i in allg.items():
            remain = i-remove
            if a not in resolved and len(remain) == 1:
                ingr = next(iter(remain))
                resolved[a] = ingr
                remove.add(ingr)
                break

    return resolved

with open('data/2020/day21.txt') as f:
    data = f.read().splitlines()

a2i, foods, allergens = parse(data)
resolved = resolve(a2i)
inert = set(flatten(foods)) - set(resolved.values())

p1 = sum(1 for i in flatten(foods) if i in inert)
print(f"{p1} ", end="")

p2 = ','.join([resolved[k] for k in sorted(resolved.keys())])
print(p2)
assert p1 == 2282
assert p2 == 'vrzkz,zjsh,hphcb,mbdksj,vzzxl,ctmzsr,rkzqs,zmhnj'

2282 vrzkz,zjsh,hphcb,mbdksj,vzzxl,ctmzsr,rkzqs,zmhnj


### Day 22: Crab Combat
https://adventofcode.com/2020/day/22

For part 2, the follwing crucial bits of problem statement apply:
            
1. The quantity of cards copied is equal to the number on the card they drew to trigger the sub-game
2. Previous rounds from other games are not considered.

In [57]:
player1 = [
    50, 19, 40, 22, 7, 4, 3, 16, 34, 45, 46, 39, 44,
    32, 20, 29, 15, 35, 41, 2, 21, 28, 6, 26, 48
]

player2 = [
    14, 9, 37, 47, 38, 27, 30, 24, 36, 31, 43, 42, 
    11, 17, 18, 10, 12, 5, 33, 25, 8, 23, 1, 13, 49
]

def play(q1, q2):
    while q1 and q2:
        c1 = q1.popleft()
        c2 = q2.popleft()

        if c1 > c2:
            q1.append(c1)
            q1.append(c2)
        else:
            q2.append(c2)
            q2.append(c1)

    return q1, q2


def score(q):
    score = 0
    for v in count(1):
        if q:
            card = q.pop()
            score += card * v
        else:
            break
    return score


def play_recursive(states, p1, p2):
    while p1 and p2:
        if (tuple(p1), tuple(p2)) in states:
            return 1
        states.add((tuple(p1), tuple(p2)))
        c1 = p1.popleft()
        c2 = p2.popleft()

        if len(p1) < c1 or len(p2) < c2:
            if c1 > c2:
                p1.append(c1)
                p1.append(c2)
            else:
                p2.append(c2)
                p2.append(c1)
        else:
            new_p1 = deque(list(p1)[:c1])  # See 1. above.
            new_p2 = deque(list(p2)[:c2])
            winner = play_recursive(set(), new_p1, new_p2)  # See 2. above.
            if winner == 1:
                p1.append(c1)
                p1.append(c2)
            else:
                p2.append(c2)
                p2.append(c1)
    if p1:
        return 1
    return 2

q1 = deque(player1)
q2 = deque(player2)
q1, q2 = play(q1, q2)
p1 = score(q1)
if p1 == 0:
    p1 = score(q2)

q1 = deque(player1)
q2 = deque(player2)
play_recursive(set(), q1, q2)

p2 = score(q1)
if p2 == 0:
    p2 = score(q2)

print(p1, p2)

assert p1 == 32083
assert p2 == 35495

32083 35495


### Day 23: Crab Cups
https://adventofcode.com/2020/day/23

In [59]:
class Cup:
    def __init__(self, label):
        self.label = label
        self.next = None

class Cups:
    def __init__(self, labels):
        self.min_label = min(labels)
        self.max_label = max(labels)
        cups = [Cup(label) for label in labels]
        self.index = {}
        for i in range(len(cups) - 1):
            cups[i].next = cups[i+1]
            self.index[cups[i].label] = cups[i]
        cups[-1].next = cups[0]
        self.index[cups[-1].label] = cups[-1]
        self.current = cups[0]
        self.start = cups[0]

    def part1(self):
        data = ''
        cur = self.index[1].next
        while cur.label != 1:
            data += str(cur.label)
            cur = cur.next
        return data

    def part2(self):
        return self.index[1].next.label * self.index[1].next.next.label

    def pick(self):
        return [
            self.current.next,
            self.current.next.next,
            self.current.next.next.next
        ]

    def play(self, rounds):
        for r in range(rounds):
            selected_cups = self.pick()
            selected_labels = [c.label for c in selected_cups]
            destination_label = self.current.label
            while True:
                destination_label -= 1
                if destination_label < self.min_label:
                    destination_label = self.max_label
                if destination_label not in selected_labels:
                    break

            # Remove the selected cups from their current location
            self.current.next = selected_cups[2].next

            # Insert selection following destination cup
            destination_cup = self.index[destination_label]
            destination_cup_next = destination_cup.next
            destination_cup.next = selected_cups[0]
            selected_cups[2].next = destination_cup_next

            # Move the selected cup on by one.
            self.current = self.current.next


labels = [5, 3, 8, 9, 1, 4, 7, 6, 2]
cups = Cups(labels)
cups.play(100)
p1 = cups.part1()
print(p1, end=' ')

labels.extend(list(range(10, 1000000 + 1)))
cups = Cups(labels)
cups.play(10000000)
p2 = cups.part2()           # ~30s
print(p2)

assert p1 == '54327968'
assert p2 == 157410423276

54327968 157410423276


### Day 24: Lobby Layout
https://adventofcode.com/2020/day/24

As always when dealing with hex grids, [Red Blob Games](https://www.redblobgames.com/grids/hexagons/) is an invaluable resource. We learnt about hex grids on [Day 11](https://adventofcode.com/2017/day/11), back in 2017, although this time the grid is "pointy top" -- that is we don't have the N-S axis. The transformation from hex grid to cube grid is described in detail [here](https://www.redblobgames.com/grids/hexagons/#neighbors-cube).

In [62]:
OFFSET = {  # x y z - cube coord offsets
    "ne": ( 1,  0, -1), "e":  ( 1, -1,  0),
    "se": ( 0, -1,  1), "sw": (-1,  0,  1),
    "w":  (-1,  1,  0), "nw": ( 0,  1, -1)
}


def parse(data):
    flippers = []
    for line in data:
        moves = []
        letters = iter(line)
        while True:
            try:
                ch = next(letters)
            except StopIteration:
                break
            if ch in 'ns':
                moves.append(f"{ch}{next(letters)}")
            else:
                moves.append(ch)
        flippers.append(moves)
        
    return flippers


def neighbours(tiles, pos):
    return sum(
        (pos[0] + delta[0], pos[1] + delta[1], pos[2] + delta[2]) in tiles
        for delta in OFFSET.values()
    )


def find_range(tiles):
    i = iter(tiles)
    mins = maxs = next(i)
    for (x, y, z) in i:
        mins = (min(x, mins[0]), min(y, mins[1]), min(z, mins[2]))
        maxs = (max(x, maxs[0]), max(y, maxs[1]), max(z, maxs[2]))
    return (mins, maxs)


def part1(data):
    tiles = {}
    for line in data:
        pos = (0, 0, 0)
        for step in line:
            delta = OFFSET[step]
            pos = (pos[0] + delta[0], pos[1] + delta[1], pos[2] + delta[2])
        tiles[pos] = not tiles.get(pos, False)
    return {k for k, v in tiles.items() if v}


def part2(state):
    for gen in range(100):
        new_state = set()
        mins, maxs = find_range(state)
        for z in range(mins[2]-1, maxs[2]+2):
            for y in range(mins[1]-1, maxs[1]+2):
                for x in range(mins[0]-1, maxs[0]+2):
                    this = (x, y, z) in state
                    n = neighbours(state, (x, y, z))
                    if this and (n == 0 or n > 2):
                        continue
                    if not this and n == 2:
                        new_state.add((x, y, z))
                        continue
                    if this:
                        new_state.add((x, y, z))
        state = new_state
    return len(state)


with open('data/2020/day24.txt') as f:
    data = f.read().splitlines()

state = part1(parse(data))
p1 = len(state)
print(p1)

p2 = part2(state)   # Takes a while....
print(p2)

assert p1 == 332
assert p2 == 3900

332
3900
